In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "natasha pritykovskaya Link Prediction") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [3]:
graphPath = "/lectures/lecture04/trainGraph"
usersToPredictPath = "/lectures/lecture04/prediction.csv"

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType(fields=[
    StructField("user", IntegerType()),
    StructField("friendsString", StringType())
])

data = spark.read.format("csv") \
        .schema(schema) \
        .option("delimiter", "\t") \
        .load(graphPath) 

In [5]:
data.show(5)

+----+--------------------+
|user|       friendsString|
+----+--------------------+
|1424|{(846,0),(1691,25...|
|4128|{(49747,0),(53568...|
|4480|{(4677,0),(22256,...|
|4656|{(520,0),(12380,0...|
|5040|{(629,0),(2471,0)...|
+----+--------------------+
only showing top 5 rows



In [6]:
data.rdd.getNumPartitions()

6

In [7]:
from pyspark.sql.functions import col, explode, collect_list, sort_array, size, split, lit


In [8]:
from pyspark.sql.functions import pandas_udf

def cutStartEndBrackets(series):
    return series.str[2:-2]

cutStartEndBracketsUDF = pandas_udf(cutStartEndBrackets, StringType())

userFriend = \
    data.select(col("user"), split(cutStartEndBracketsUDF(col("friendsString")), "\),\(").alias("friendsMasks"))\
    .withColumn("friendMask", explode('friendsMasks'))\
    .withColumn("friend", split(col("friendMask"), ",")[0])\
    .select(col("user"), col("friend").cast("integer"))

<img src="pics/step1.png" width=700/>

In [9]:
usersWithCommonFriend = userFriend\
    .groupBy("friend")\
    .agg(collect_list("user").alias("usersWithCommonFriend")) \
    .select("usersWithCommonFriend")\
    .where(size(col("usersWithCommonFriend")) >= 2)\
    .select(sort_array("usersWithCommonFriend").alias("sortedUsersWithCommonFriend"))\
    .drop("usersWithCommonFriend")

<img src="pics/step2.png" width=700/>
<img src="pics/step3.png" width=700/>


In [10]:
usersWithCommonFriend.show()

+---------------------------+
|sortedUsersWithCommonFriend|
+---------------------------+
|          [131318, 2038934]|
|       [11979968, 11979968]|
|       [4471643, 4864911...|
|           [39812, 3996243]|
|       [34660, 960407, 1...|
|         [215426, 10989590]|
|       [294833, 3541819,...|
|           [585940, 595464]|
|         [7941296, 7941296]|
|       [213875, 3310959,...|
|       [1406846, 2044978...|
|       [1208384, 1208384...|
|        [2625623, 16545603]|
|        [9787095, 11510201]|
|       [1093525, 1543499...|
|       [10599936, 10599936]|
|       [5021715, 5262480...|
|        [1651903, 10655397]|
|       [1919, 582013, 35...|
|         [3243642, 5845843]|
+---------------------------+
only showing top 20 rows



In [11]:
schema_users_to_pred = StructType(fields=[
    StructField("user", IntegerType()),
])

usersToPredict = spark.read.format("csv") \
    .schema(schema) \
    .load(usersToPredictPath) \
    .select(col("user").cast("integer")) \
    .rdd.map(lambda t : t.user).collect()

usersToPredictBC = spark.sparkContext.broadcast(set(usersToPredict))

In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType

def pairsWithCommonFriend(usersWithCommonFriend):
    pairs = []
    for user1Index in range(0, len(usersWithCommonFriend)):
        for user2Index in range(user1Index + 1, len(usersWithCommonFriend)):
            if user1Index != user2Index:
                if (usersWithCommonFriend[user1Index] in usersToPredictBC.value or \
                usersWithCommonFriend[user2Index] in usersToPredictBC.value):
                    pairs.append((usersWithCommonFriend[user1Index], usersWithCommonFriend[user2Index]))
    return pairs

schema = ArrayType(ArrayType(IntegerType()))

pairsWithCommonFriendUdf = udf(pairsWithCommonFriend, schema)

<img src="pics/step4_2.png" width=700/>

In [13]:
from pyspark.sql.functions import pandas_udf
import pandas as pd

def pairsWithCommonFriend(series):
    pairs_lists = []
    for usersWithCommonFriend in series:
        pairs = []
        for user1Index in range(0, len(usersWithCommonFriend)):
            for user2Index in range(user1Index + 1, len(usersWithCommonFriend)):
                if user1Index != user2Index:
                    if usersWithCommonFriend[user1Index] in usersToPredictBC.value or \
                    usersWithCommonFriend[user2Index] in usersToPredictBC.value:
                        pairs.append((usersWithCommonFriend[user1Index], usersWithCommonFriend[user2Index]))
        pairs_lists.append(pairs)
    return pd.Series(pairs_lists)
        
pairsWithCommonFriendUdf = pandas_udf(pairsWithCommonFriend, schema)

commonFriendsCounts = usersWithCommonFriend\
            .select(pairsWithCommonFriendUdf("sortedUsersWithCommonFriend").alias("pairsWithCommonFriend"))\
            .where(size(col("pairsWithCommonFriend")) > 0)    

commonFriendsCounts\
    .withColumn("pairWithCommonFriend", explode("pairsWithCommonFriend"))\
    .drop(col("pairsWithCommonFriend"))\
    .groupBy(col("pairWithCommonFriend"))\
    .count()\
    .show(20)

+--------------------+-----+
|pairWithCommonFriend|count|
+--------------------+-----+
|   [340595, 2353061]|    2|
|[12228454, 13773063]|   59|
|   [608428, 2724603]|    2|
|   [995605, 5126964]|    1|
|   [971590, 9074160]|    2|
|  [658247, 16584168]|  726|
|[10610321, 12007797]|   16|
|   [912457, 2514264]|    5|
|  [850736, 14118378]|   48|
| [1087903, 16519367]|    1|
|  [1802940, 2995755]|    1|
|  [2280725, 3192528]|   64|
|  [3233402, 8128284]|   19|
|  [461266, 11738162]|    2|
|    [724636, 943158]|    1|
|   [185916, 3792180]|    5|
|   [434247, 3792180]|    1|
|    [411957, 904004]|    6|
|  [825709, 11243712]|  132|
| [4626992, 10033766]|    6|
+--------------------+-----+
only showing top 20 rows



In [ ]:
def pairsWithCommonFriend(series):
    pairs_lists = []
    for usersWithCommonFriend in series:
        pairs = []
        for user1Index in range(0, len(usersWithCommonFriend)):
            for user2Index in range(user1Index + 1, len(usersWithCommonFriend)):
                if user1Index != user2Index:
                    pairs.append((usersWithCommonFriend[user1Index], usersWithCommonFriend[user2Index]))
        pairs_lists.append(pairs)
    return pd.Series(pairs_lists)
         
pairsWithCommonFriendUdf = pandas_udf(pairsWithCommonFriend, schema)

commonFriendsCounts = usersWithCommonFriend\
            .select(pairsWithCommonFriendUdf("sortedUsersWithCommonFriend").alias("pairsWithCommonFriend"))\
            .where(size(col("pairsWithCommonFriend")) > 0)    

commonFriendsCounts\
    .withColumn("pairWithCommonFriend", explode("pairsWithCommonFriend"))\
    .drop(col("pairsWithCommonFriend"))\
    .groupBy(col("pairWithCommonFriend"))\
    .count()\
    .show(20)

In [ ]:
from functools import partial

def pairsWithCommonFriendUpgraded(series, modulo):
    pairs_lists = []

    for usersWithCommonFriend in series:
        pairs = []
        for user1Index in range(0, len(usersWithCommonFriend)):
             for user2Index in range(user1Index + 1, len(usersWithCommonFriend)):
                    if user1Index != user2Index and user1Index % 13 == modulo:
                        pairs.append((usersWithCommonFriend[user1Index], usersWithCommonFriend[user2Index]))
        pairs_lists.append(pairs)
    return pd.Series(pairs_lists)


for i in range(13):
    pairsWithCommonFriendUdfUpgraded = pandas_udf(partial(pairsWithCommonFriendUpgraded, modulo=i), schema)

    commonFriendsCounts = usersWithCommonFriend\
            .select(pairsWithCommonFriendUdfUpgraded("sortedUsersWithCommonFriend").alias("pairsWithCommonFriend"))\
            .where(size(col("pairsWithCommonFriend")) > 0)\
            .write.parquet("pairs/" + str(i), mode = "overwrite")

In [ ]:
spark.read.parquet("pairs/*").count()

In [ ]:
spark.read.parquet("pairs/0").count()

In [27]:
spark.read.parquet("pairs/*")\
    .withColumn("pairWithCommonFriend", explode("pairsWithCommonFriend"))\
    .drop(col("pairsWithCommonFriend"))\
    .groupBy(col("pairWithCommonFriend"))\
    .count()\
    .show(5)

+--------------------+-----+
|pairWithCommonFriend|count|
+--------------------+-----+
| [1481111, 15756137]|    1|
| [8039479, 10812715]|    4|
| [9102327, 16272182]|    1|
|   [709520, 1563367]|    2|
|    [200736, 228410]|    2|
+--------------------+-----+
only showing top 5 rows



In [ ]:
spark.stop()